In [1]:
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import eigsh
from scipy import sparse

hamiltonian = np.zeros((6240,6240))

with open('datafile','r') as data:
    count = 0
    while True:
        line = data.readline()
        if count == 0:
            count += 1
        if not line:
            break
        else:
            values = line.split()
            hamiltonian[int(values[0])-1][int(values[1])-1] = float(values[2])
            hamiltonian[int(values[1])-1][int(values[0])-1] = float(values[2])

In [2]:
size = 100
sub_hamiltonian = hamiltonian[0:size,0:size] # used 100 particles to work with
sparse_sub_hamiltonian = sparse.csr_matrix(sub_hamiltonian)
eigenvalues, eigenvectors = eigsh(sparse_sub_hamiltonian,99,which = 'LM')
print sum(eigenvalues) # should be -8234.4466281 

-8234.4466281


In [72]:
# arbitrarily set one of the determinants to 1.
psi_initial = np.zeros(shape=(size, 1), dtype = complex)
psi_initial[0] = 1 # I set the first determinant to 1

In [154]:
def runge_kutta(t1,dt):
    """
    Parameters: total time t1,time step dt
    Returns a list of the sum_probabilities at each time step
    Implemented based on http://lpsa.swarthmore.edu/NumInt/NumIntFourth.html 
    """
    global psi_initial
    global size
    psi_prev_rk = psi_initial
    psi_next = np.zeros(shape=(size,1),dtype = complex)
    k1 = np.zeros(shape=(size,1),dtype = complex)
    k2 = np.zeros(shape=(size,1),dtype = complex)
    k3 = np.zeros(shape=(size,1),dtype = complex)
    k4 = np.zeros(shape=(size,1),dtype = complex)
    y1 = np.zeros(shape=(size,1),dtype = complex)
    y2 = np.zeros(shape=(size,1),dtype = complex)
    y3 = np.zeros(shape=(size,1),dtype = complex)
    y = []
    index = int(t1/dt)
    for i in range(index):
        # first loop to update k1, y1
        for i in range(size):
            accumulator = sub_hamiltonian[i][i]*y1[i]
            for j in range(size):
                if i != j:
                    accumulator += sub_hamiltonian[i][j]*psi_prev_rk[j]
            k1[i] = accumulator*1j
            y1[i] = psi_prev_rk[i] + accumulator*(dt/6)*1j
            accumulator = 0 # reset accumulator to 0
        # To get k2, y2
        assert accumulator == 0
        for i in range(size):
            accumulator = sub_hamiltonian[i][i]*y1[i]
            for j in range(size):
                if i != j:
                    accumulator += sub_hamiltonian[i][j]*y1[j]
            y2[i] = y1[i] + accumulator*(dt/3)*1j
            k2[i] = accumulator*(dt/3)*1j
            accumulator = 0 # reset accumulator to 0
        # To get k3, y3
        assert accumulator == 0
        for i in range(size):
            accumulator = sub_hamiltonian[i][i]*y2[i]
            for j in range(size):
                if i != j:
                    accumulator += sub_hamiltonian[i][j]*y2[j]
            k3[i] = accumulator*(dt/3)*1j
            y3[i] = y2[i] + accumulator*(dt/3)*1j
            accumulator = 0 # reset accumulator to 0
        # To get k4, y4
        assert accumulator == 0
        for i in range(size):
            accumulator = sub_hamiltonian[i][i]*y3[i]
            for j in range(size):
                if i != j:
                    accumulator += sub_hamiltonian[i][j]*y3[j]
            k4[i] = accumulator*1j*(dt/6)
            accumulator = 0 # reset accumulator to 0
            psi_next[i] = psi_prev_rk[i] + (k1[i]+ k2[i]+ k3[i]+k4[i])
        psi_prev_rk = psi_next
        psi_next = np.zeros(shape=(size,1),dtype = complex)
        # For plotting
        y.append(sum((np.absolute(psi_prev_rk)**2)))
    return y

In [157]:
def frange(start, stop, step):
    i = start
    while i < stop:
             yield i
             i += step

In [168]:
import matplotlib.pyplot as plt
list_dt = [10**x for x in range(-3,1)]
probabilities = []
for dt in list_dt:
    y = runge_kutta(10,dt)
    prob = y.pop()
    probabilities.append(prob)

In [167]:
print probabilities # The sum of probabilities exceeds the maximum value for time steps of 10^(-3) and 10^(-2)
# The time step of 10^(-1) results in a much higher divergence in probabilities than a time step of 1 for total time,10

[array([ nan]), array([ nan]), array([  3.27703921e+264]), array([  4.00708172e+19])]
